# Number of Simulations vs. Pareto Ranks by Team

This notebook explores the number of simulations that were done by team and grouped by 
their experimental condition and plots them against their Pareto ranks.

In [74]:
import sys
import os.path

sys.path.append("../CommonModules") # go to parent dir/CommonModules

import Learning2019GTL.Globals as Globals
import Learning2019GTL.DataConnector as DataConnector

data_map = Globals.FileNameMaps()

TEAM_MAP = dict([[v,k] for k,v in data_map.CSV_MAP.items()])

DATA_FILE = os.path.expanduser("~/Learning2019Data/PostProcessingResults/TeamNumSimulationsByFile.csv")
DATA_FILE_PARETO = os.path.expanduser("~/Learning2019Data/PostProcessingResults/ParetoRankingByFile.csv")

import csv
import numpy as np

conditions = ['A - control (discuss conference) at beginning', 'B - strategy at beginning', 'D - strategy at mid']

class AnalyzedData:
    team_str   = ''
    unique_sims = -1
    total_sims  = -1
    condition  = ''
    pareto_ranks_coarse = -1 # one or more solutions to be the same rank
    pareto_ranks_fine   = -1 # more solutions == better rank
    
analyzed_data = {} # empty dictionary

with open(DATA_FILE, 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        team_file = row['File']
        team_str = TEAM_MAP[team_file]
        t = DataConnector.Team()
        condition = t.getCondition(team_str)[0]
        #print(row)
        #print(condition
        if team_str in analyzed_data.keys():
            datum = analyzed_data[team_str]
        else:
            datum = AnalyzedData()
            analyzed_data[team_str] = datum
            datum.team_str = team_str
            
        datum.condition = condition
        datum.unique_sims = int(row[' UniqueSims'])
        datum.total_sims = int(row[' TotalSims'])
        


## Checking that I can get back data grouped by conditions for plotting

In [75]:
uniqueSimsA = [analyzed_data[k].unique_sims for k in analyzed_data.keys() if analyzed_data[k].condition == 'A' ]
uniqueSimsB = [analyzed_data[k].unique_sims for k in analyzed_data.keys() if analyzed_data[k].condition == 'B' ]
uniqueSimsD = [analyzed_data[k].unique_sims for k in analyzed_data.keys() if analyzed_data[k].condition == 'D' ]

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt


font = {'family' : 'DejaVu Sans',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)

golden_ratio = np.array([1.61803398875, 1])
plt.figure(figsize=golden_ratio*8)

uniqueSimsA = np.array(uniqueSimsA)
uniqueSimsB = np.array(uniqueSimsB)
uniqueSimsD = np.array(uniqueSimsD)

plt.hist(uniqueSimsA-1/6, bins=np.arange(0, 18, 1/6), density=False)
plt.hist(uniqueSimsB+0/6, bins=np.arange(0, 18, 1/6), density=False)
plt.hist(uniqueSimsD+1/6, bins=np.arange(0, 18, 1/6), density=False)
plt.xlim([0, 18])
plt.legend(conditions, fontsize=12, loc=2)
plt.ylabel('Number of Teams')
plt.xlabel('Number of Unique Simulations')
plt.show()


## Loading Pareto Ranks

In [76]:
def getPRank(row):
    rank = -1
    for p in range(17):
        key = f'P{p}'
        if int(row[key]) > 0:
            rank = p
            break
            
    return rank

def getPRankWithNum(row):
    rank = -1
    for p in range(17):
        key = f'P{p}'
        val = int(row[key])
        if val > 0:
            rank = p + 1/(val + 1)
            break
            
    return rank

with open(DATA_FILE_PARETO, 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        team_file = row['File']
        team_str = TEAM_MAP[team_file]
        rank_coarse = getPRank(row)
        rank_fine   = getPRankWithNum(row)
        datum = analyzed_data[team_str] # we should have all the data points in the number of sims loop    
        datum.pareto_ranks_coarse = rank_coarse
        datum.pareto_ranks_fine   = rank_fine

## Number of unique simulations verses pareto ranks

In [82]:
paretoRanksA = [analyzed_data[k].pareto_ranks_fine for k in analyzed_data.keys() if analyzed_data[k].condition == 'A' ]
paretoRanksB = [analyzed_data[k].pareto_ranks_fine for k in analyzed_data.keys() if analyzed_data[k].condition == 'B' ]
paretoRanksD = [analyzed_data[k].pareto_ranks_fine for k in analyzed_data.keys() if analyzed_data[k].condition == 'D' ]

allData = paretoRanksA + paretoRanksB + paretoRanksD

import scipy.stats
rankedData = scipy.stats.rankdata(allData, method='dense')
paretoRanksA = rankedData[0:len(paretoRanksA)]
paretoRanksB = rankedData[len(paretoRanksA):len(paretoRanksA)+len(paretoRanksB)]
paretoRanksD = rankedData[len(paretoRanksA)+len(paretoRanksB):]

plt.figure(figsize=golden_ratio*8)
plt.plot(uniqueSimsA, paretoRanksA, marker="s", linewidth=0, markersize=15)
plt.plot(uniqueSimsB, paretoRanksB, marker="o", linewidth=0, markersize=15)
plt.plot(uniqueSimsD, paretoRanksD, marker="d", linewidth=0, markersize=15)
plt.xlabel("Number of Unique Soltuions")
plt.ylabel("Pareto Ranking (lower is better)")
plt.legend(conditions, fontsize=16, loc='upper left', bbox_to_anchor=(0,1.25))
plt.show()

